**New way**


#
3
3

3
3 :)

> Helloo lets start again





In [7]:
# Mount Google Drive (with force_remount to ensure fresh mount)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define paths based on folder contents
import os
CHECKPOINTS_PATH = '/content/drive/My Drive/Checkpoints/'  # Correct folder
VIDEO_PATH = os.path.join(CHECKPOINTS_PATH, 'Video_input.mp4')  # Updated file name
CHECKPOINT_PATH = os.path.join(CHECKPOINTS_PATH, 'Copy of Wav2Lip-SD-GAN.pt')  # Updated file name

# Check if the checkpoints folder exists
if os.path.exists(CHECKPOINTS_PATH):
    print(f"Checkpoints folder found at: {CHECKPOINTS_PATH}")
else:
    print(f"Checkpoints folder NOT FOUND at: {CHECKPOINTS_PATH}")
    print("Please verify the folder path. Possible locations:")
    !find "/content/drive/My Drive" -type d -name "Checkpoints"
    !find "/content/drive/My Drive" -type d -name "checkpoints"
    !find "/content/drive/My Drive" -type d -iname "*checkpoint*"
    print("If the correct folder is found above, update CHECKPOINTS_PATH to the correct path.")
    print("If not found, check Google Drive manually.")

# Verify files
print("\nVideo exists:", os.path.exists(VIDEO_PATH))
print("Checkpoint exists:", os.path.exists(CHECKPOINT_PATH))

# List folder contents
if os.path.exists(CHECKPOINTS_PATH):
    print("Files in checkpoints folder:", os.listdir(CHECKPOINTS_PATH))
else:
    print("Cannot list files: Checkpoints folder does not exist.")

Mounted at /content/drive
Checkpoints folder found at: /content/drive/My Drive/Checkpoints/

Video exists: True
Checkpoint exists: True
Files in checkpoints folder: ['Input_video.mp4', 'Copy of .DS_Store', 'Copy of Wav2Lip-SD-GAN.pt', 'Copy of Wav2Lip-SD-NOGAN.pt', 'Video_input.mp4', 'Copy of Video_input.mp4', 'extracted_audios.wav']


In [8]:
# Clone Wav2Lip repository
!git clone https://github.com/Rudrabha/Wav2Lip.git
!mkdir -p Wav2Lip/checkpoints

# Copy checkpoint to Wav2Lip directory
!cp "{CHECKPOINT_PATH}" "Wav2Lip/checkpoints/wav2lip_gan.pth"
print("Checkpoint copied to Wav2Lip/checkpoints:", os.path.exists('Wav2Lip/checkpoints/wav2lip_gan.pth'))

# Verify checkpoint file
if os.path.exists('Wav2Lip/checkpoints/wav2lip_gan.pth'):
    print("Checkpoint successfully set up for Wav2Lip.")
else:
    print("Checkpoint copy failed. Please check CHECKPOINT_PATH and try uploading manually.")
    from google.colab import files
    uploaded = files.upload()  # Upload Copy of Wav2Lip-SD-GAN.pt
    !mv "Copy of Wav2Lip-SD-GAN.pt" "Wav2Lip/checkpoints/wav2lip_gan.pth"
    print("Manual upload completed:", os.path.exists('Wav2Lip/checkpoints/wav2lip_gan.pth'))

fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
Checkpoint copied to Wav2Lip/checkpoints: True
Checkpoint successfully set up for Wav2Lip.


In [9]:
!pip install numpy==1.24.4

In [10]:
!pip install librosa==0.8.0

In [14]:
!pip install librosa --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.0
    Uninstalling librosa-0.8.0:
      Successfully uninstalled librosa-0.8.0


In [15]:
# Install compatible dependencies
!pip install opencv-contrib-python==4.11.0.86 torch torchvision gTTS pydub SpeechRecognition deep-translator moviepy numba==0.60.0 llvmlite==0.43.0

# Install Wav2Lip requirements, excluding problematic versions
with open('Wav2Lip/requirements.txt', 'r') as f:
    requirements = f.readlines()
filtered_requirements = [req for req in requirements if not any(x in req for x in ['opencv-python==4.1.0.25', 'numpy==1.17.1', 'librosa==0.7.0', 'torch==1.1.0', 'torchvision==0.3.0', 'numba==0.48', 'llvmlite<0.32.0'])]
filtered_requirements = [req.replace('tqdm==4.45.0', 'tqdm>=4.45.0') for req in filtered_requirements]
with open('Wav2Lip/filtered_requirements.txt', 'w') as f:
    f.writelines(filtered_requirements)
!pip install -r Wav2Lip/filtered_requirements.txt

# Verify installations
import importlib
required_libraries = ['librosa', 'numpy', 'cv2', 'torch', 'torchvision', 'gtts', 'pydub', 'speech_recognition', 'deep_translator', 'moviepy', 'numba']
for lib in required_libraries:
    try:
        importlib.import_module(lib)
        print(f"{lib} is installed.")
    except Exception as e:
        print(f"{lib} failed to import: {e}")

librosa is installed.
numpy is installed.
cv2 is installed.
torch is installed.
torchvision is installed.
gtts is installed.
pydub is installed.
speech_recognition is installed.
deep_translator is installed.
moviepy is installed.
numba is installed.


In [16]:
import librosa
import numpy
import cv2
import tqdm

print("✅ Library versions:")
print(f"librosa version: {librosa.__version__}")
print(f"numpy version: {numpy.__version__}")
print(f"cv2 version: {cv2.__version__}")
print(f"tqdm version: {tqdm.__version__}")

''' Library versions:
librosa version: 0.10.2
numpy version: 1.25.2
cv2 version: 4.11.0
tqdm version: 4.67.1
'''



✅ Library versions:
librosa version: 0.8.0
numpy version: 1.24.4
cv2 version: 4.11.0
tqdm version: 4.67.1


' Library versions:\nlibrosa version: 0.10.2\nnumpy version: 1.25.2\ncv2 version: 4.11.0\ntqdm version: 4.67.1\n'

In [17]:
from moviepy.editor import VideoFileClip
import os

# Ensure VIDEO_PATH and CHECKPOINTS_PATH are defined
VIDEO_PATH = '/content/drive/My Drive/Checkpoints/Input_video.mp4'
CHECKPOINTS_PATH = '/content/drive/My Drive/Checkpoints/'

# Verify video file exists
if not os.path.exists(VIDEO_PATH):
    print(f"Video file not found at: {VIDEO_PATH}")
    print("Please verify VIDEO_PATH or upload the video:")
    from google.colab import files
    uploaded = files.upload()
    !mv "Video_input.mp4" "{CHECKPOINTS_PATH}"
else:
    print(f"Video file found at: {VIDEO_PATH}")

# Extract audio
try:
    video = VideoFileClip(VIDEO_PATH)
    audio_path = os.path.join(CHECKPOINTS_PATH, 'extracted_audios.wav')
    video.audio.write_audiofile(audio_path)
    video.close()
    print("Audio extracted to:", audio_path)
    print("Audio file exists:", os.path.exists(audio_path))
except Exception as e:
    print(f"Error extracting audio: {e}")

Video file found at: /content/drive/My Drive/Checkpoints/Input_video.mp4
MoviePy - Writing audio in /content/drive/My Drive/Checkpoints/extracted_audios.wav


MoviePy - Done.
Audio extracted to: /content/drive/My Drive/Checkpoints/extracted_audios.wav
Audio file exists: True


In [18]:
import os
CHECKPOINTS_PATH = '/content/drive/My Drive/Checkpoints/'
audio_path = os.path.join(CHECKPOINTS_PATH, 'extracted_audios.wav')
output_path = os.path.join(CHECKPOINTS_PATH, 'lip_synced_output.mp4')
!python Wav2Lip/inference.py --checkpoint_path Wav2Lip/checkpoints/wav2lip_gan.pth --face "{VIDEO_PATH}" --audio "{audio_path}" --outfile "{output_path}"
print("Lip-synced video saved to:", output_path)
print("Output file exists:", os.path.exists(output_path))

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 1113
Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 280, in <module>
    main()
  File "/content/Wav2Lip/inference.py", line 225, in main
    mel = audio.melspectrogram(wav)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Wav2Lip/audio.py", line 47, in melspectrogram
    S = _amp_to_db(_linear_to_mel(np.abs(D))) - hp.ref_level_db
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Wav2Lip/audio.py", line 95, in _linear_to_mel
    _mel_basis = _build_mel_basis()
                 ^^^^^^^^^^^^^^^^^^
  File "/content/Wav2Lip/audio.py", line 100, in _build_mel_basis
    return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: mel() takes 0 positional arguments but 2 positional arguments (and 3 keyword-only arguments) were given
Lip-synced video saved t

In [ ]:
yy# Patch inference.py to lower face detection threshold
!cp Wav2Lip/inference.py Wav2Lip/inference.py.bak
with open('Wav2Lip/inference.py', 'r') as file:
    lines = file.readlines()
with open('Wav2Lip/inference.py', 'w') as file:
    for line in lines:
        if 'det_batch = detector.detect_faces(frame, conf_th=0.8)' in line:
            file.write(line.replace('conf_th=0.8', 'conf_th=0.6'))
        else:
            file.write(line)
!grep "conf_th" Wav2Lip/inference.py

import os
# Define paths
CHECKPOINTS_PATH = '/content/drive/My Drive/Checkpoints/'
VIDEO_PATH = os.path.join(CHECKPOINTS_PATH, 'Video_input.mp4')
AUDIO_PATH = os.path.join(CHECKPOINTS_PATH, 'extracted_audios.wav')
CHECKPOINT_PATH = 'Wav2Lip/checkpoints/wav2lip_gan.pth'
OUTPUT_PATH = os.path.join(CHECKPOINTS_PATH, 'lip_synced_output.mp4')

# Verify inputs
print("Video exists:", os.path.exists(VIDEO_PATH))
print("Audio exists:", os.path.exists(AUDIO_PATH))
print("Checkpoint exists:", os.path.exists(CHECKPOINT_PATH))
if not all([os.path.exists(VIDEO_PATH), os.path.exists(AUDIO_PATH), os.path.exists(CHECKPOINT_PATH)]):
    print("One or more input files are missing. Please check paths or upload missing files:")
    from google.colab import files
    uploaded = files.upload()
    if 'Input_video.mp4' in uploaded:
        !mv "Input_video.mp4" "{CHECKPOINTS_PATH}"
    if 'extracted_audios.wav' in uploaded:
        !mv "extracted_audios.wav" "{CHECKPOINTS_PATH}"
    if 'wav2lip_gan.pth' in uploaded:
        !mv "wav2lip_gan.pth" "Wav2Lip/checkpoints/"
    print("Uploaded files moved. Re-check paths:")
    print("Video exists:", os.path.exists(VIDEO_PATH))
    print("Audio exists:", os.path.exists(AUDIO_PATH))
    print("Checkpoint exists:", os.path.exists(CHECKPOINT_PATH))

# Run Wav2Lip inference
try:
    !python Wav2Lip/inference.py --checkpoint_path "{CHECKPOINT_PATH}" --face "{VIDEO_PATH}" --audio "{AUDIO_PATH}" --outfile "{OUTPUT_PATH}"
    print("Lip-synced video saved to:", OUTPUT_PATH)
    print("Output file exists:", os.path.exists(OUTPUT_PATH))
except Exception as e:
    print(f"Error running Wav2Lip inference: {e}")

Video exists: True
Audio exists: True
Checkpoint exists: True
Using cuda for inference.
Reading video frames...
^C
Lip-synced video saved to: /content/drive/My Drive/Checkpoints/lip_synced_output.mp4
Output file exists: False


In [ ]:
# Backup inference.py
!cp Wav2Lip/inference.py Wav2Lip/inference.py.bak

# Patch inference.py to lower face detection threshold
with open('Wav2Lip/inference.py', 'r') as file:
    lines = file.readlines()
with open('Wav2Lip/inference.py', 'w') as file:
    for line in lines:
        # Adjust confidence threshold in face detection (default is 0.8)
        if 'det_batch = detector.detect_faces(frame, conf_th=0.8)' in line:
            file.write(line.replace('conf_th=0.8', 'conf_th=0.6'))
        else:
            file.write(line)

# Optional: Verify patch
!grep "conf_th" Wav2Lip/inference.py

cp: cannot stat 'Wav2Lip/inference.py': No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'Wav2Lip/inference.py'

In [ ]:
import os

# Define paths
CHECKPOINTS_PATH = '/content/drive/My Drive/Checkpoints/'
VIDEO_PATH = os.path.join(CHECKPOINTS_PATH, 'Video_input.mp4')
AUDIO_PATH = os.path.join(CHECKPOINTS_PATH, 'extracted_audios.wav')
CHECKPOINT_PATH = 'Wav2Lip/checkpoints/wav2lip_gan.pth'
OUTPUT_PATH = os.path.join(CHECKPOINTS_PATH, 'lip_synced_output.mp4')

# Verify inputs
print("Video exists:", os.path.exists(VIDEO_PATH))
print("Audio exists:", os.path.exists(AUDIO_PATH))
print("Checkpoint exists:", os.path.exists(CHECKPOINT_PATH))
if not all([os.path.exists(VIDEO_PATH), os.path.exists(AUDIO_PATH), os.path.exists(CHECKPOINT_PATH)]):
    print("One or more input files are missing. Please check paths or upload missing files:")
    from google.colab import files
    uploaded = files.upload()
    if 'Video_input.mp4' in uploaded:
        !mv "Video_input.mp4" "{CHECKPOINTS_PATH}"
    if 'extracted_audios.wav' in uploaded:
        !mv "extracted_audios.wav" "{CHECKPOINTS_PATH}"
    if 'wav2lip_gan.pth' in uploaded:
        !mv "wav2lip_gan.pth" "Wav2Lip/checkpoints/"
    print("Uploaded files moved. Re-check paths:")
    print("Video exists:", os.path.exists(VIDEO_PATH))
    print("Audio exists:", os.path.exists(AUDIO_PATH))
    print("Checkpoint exists:", os.path.exists(CHECKPOINT_PATH))

# Verify libraries
!pip install librosa==0.8.0 numpy==1.24.4 numba==0.60.0 llvmlite==0.43.0 opencv-contrib-python==4.11.0.86
import importlib
for lib in ['librosa', 'numpy', 'numba', 'cv2']:
    try:
        module = importlib.import_module(lib)
        print(f"{lib} version: {module.__version__}")
    except Exception as e:
        print(f"Failed to import {lib}: {e}")

# Verify audio.py
!grep "librosa.filters.mel" Wav2Lip/audio.py

# Patch inference.py to lower face detection threshold
!cp Wav2Lip/inference.py Wav2Lip/inference.py.bak
with open('Wav2Lip/inference.py', 'r') as file:
    lines = file.readlines()
with open('Wav2Lip/inference.py', 'w') as file:
    for line in lines:
        if 'det_batch = detector.detect_faces(frame, conf_th=0.8)' in line:
            file.write(line.replace('conf_th=0.8', 'conf_th=0.6'))
        else:
            file.write(line)
!grep "conf_th" Wav2Lip/inference.py

# Run Wav2Lip inference with resize_factor for CUDA
try:
    !python Wav2Lip/inference.py --checkpoint_path "{CHECKPOINT_PATH}" --face "{VIDEO_PATH}" --audio "{AUDIO_PATH}" --outfile "{OUTPUT_PATH}" --resize_factor 2
    print("Lip-synced video saved to:", OUTPUT_PATH)
    print("Output file exists:", os.path.exists(OUTPUT_PATH))
except Exception as e:
    print(f"Error running Wav2Lip inference: {e}")

Video exists: True
Audio exists: True
Checkpoint exists: True
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.5 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201361 sha256=14321ad07baa3f77728c3809378874bfede842055b2ef0167c6be1717293d8d7
  Stored in directory: /root/.cache/pip/wheels/8a/3a/85/2accb8befd6aecdb81eb4f54a856466e1e59d294a41a0bad73
Successfully built librosa
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0
    Uninstalling librosa-0.11.0:
      Successfully uninstalled librosa-0.11.0
ERROR: pip's dependency resolver does not currently take into a

librosa version: 0.11.0
numpy version: 1.25.2
numba version: 0.60.0
cv2 version: 4.11.0
    return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 3, in <module>
    import scipy, cv2, os, sys, argparse, audio
  File "/content/Wav2Lip/audio.py", line 1, in <module>
    import librosa
  File "/usr/local/lib/python3.11/dist-packages/librosa/__init__.py", line 211, in <module>
    from . import core
  File "/usr/local/lib/python3.11/dist-packages/librosa/core/__init__.py", line 9, in <module>
    from .constantq import *  # pylint: disable=wildcard-import
    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/librosa/core/constantq.py", line 1058, in <module>
    dtype=np.complex,
          ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/numpy/__init__.py", line 305, in __getattr__
    raise AttributeError(__former_attrs__[attr])
AttributeError: module 'numpy'